In [29]:
# Builds a database corpus_NLP_preprocessed, which contains a table for each label containing entries
# (ID_Post, Word-Embeding-Array of text, Value-of-label)
# Important: To read this database, you need to read the array type from the database, or else SQL won't
# know what that is, i.e. call the connection using
# con = sqlite3.connect("corpus_NLP_preprocessed", detect_types=sqlite3.PARSE_DECLTYPES)

# IMPORTANT: You have to manually transfer the Table 'Posts' from corpus.sqlite3
# to corpus_NLP_preprocessed.sqlite3


import spacy
import sqlite3
import pandas as pd
import numpy as np
import io

def adapt_array(arr):
    """
    http://stackoverflow.com/a/31312102/190597 (SoulNibbler)
    """
    out = io.BytesIO()
    np.save(out, arr)
    out.seek(0)
    return sqlite3.Binary(out.read())

def convert_array(text):
    out = io.BytesIO(text)
    out.seek(0)
    return np.load(out)


# Converts np.array to TEXT when inserting
sqlite3.register_adapter(np.ndarray, adapt_array)

# Converts TEXT to np.array when selecting
sqlite3.register_converter("array", convert_array)

x = np.arange(12).reshape(2,6)

con = sqlite3.connect("corpus_NLP_preprocessed", detect_types=sqlite3.PARSE_DECLTYPES)
cnx = con.cursor()

con_old_db =  sqlite3.connect("corpus.sqlite3")


nlp = spacy.load("de_core_news_lg")



In [30]:
def getLabeledPart(label):
    """

    :param label: Das Label, so dass wir alle Posts auswählen, bei denen dieses Label gesetzt ist
    :return: Ein DataFrame mit dem Index ID_Post (als 0. Spalte) und den Spalten 'Text' sowie 'Value'.
             'Value' besagt hierbei für jeden Post, ob das Label zutrifft oder nicht (1 oder 0)
    """
    value_count = pd.read_sql_query("SELECT * "
                                    "FROM Annotations_consolidated "
                                    "WHERE Category = '{}'".format(label),con_old_db).shape[0]

    # Wir holen uns alle Posts, die gemäß dem Parameter label gelabeled sind
    wanted_join_table = pd.read_sql_query(
        "Select ID_Post, Headline, Body, Value "
        "FROM Posts INNER JOIN Annotations_consolidated USING(ID_Post)"
        "WHERE Annotations_consolidated.Category = '{}'".format(label),con_old_db)

    assert wanted_join_table.shape[0] == value_count

    # Setzen ID_Post als Index des DataFrames
    wanted_join_table = wanted_join_table.set_index("ID_Post")

    # Fusen von Headline & Body
    for row_ID, cur_row in wanted_join_table.iterrows():
        headline = cur_row["Headline"]
        body = cur_row["Body"]
        if not headline:
            wanted_join_table.loc[row_ID,"Text"] = cur_row["Body"]
        elif not body:
            wanted_join_table.loc[row_ID,"Text"] = cur_row["Headline"]
        else:
            wanted_join_table.loc[row_ID,"Text"] = cur_row["Headline"] +"\n"+ cur_row["Body"]

    wanted_join_table = wanted_join_table.drop("Headline",axis=1).drop("Body",axis=1)

    return wanted_join_table


In [31]:
labels = ["ArgumentsUsed","Discriminating","Inappropriate","OffTopic","PersonalStories","PossiblyFeedback","SentimentNegative","SentimentNeutral","SentimentPositive"]



In [32]:
# table_names = pd.read_sql_query("SELECT * from sqlite_master", con)
#
# for label in labels:
#     if not label in table_names[["name"]].values:
#         cnx.execute("""
#         drop table {0}
#         """.format(label))


In [ ]:
# Written so it is executed only once. If during a run an error occurs, reset all tables
# (commented out code in last cell)
from sqlite3 import IntegrityError

table_names = pd.read_sql_query("SELECT * from sqlite_master", con)
for label in labels:
    if not label in table_names[["name"]].values:
        cnx.execute("""
        create table {0}
        (
            ID_Post  integer
        constraint {0}_pk
        primary key,
        Text_Array array,
        Value    integer
        );
        """.format(label))

    for label in labels:
        # Table_Join holen:
        joined_table = getLabeledPart(label)

        for index, element in joined_table[["Text"]].iterrows():
            vector = nlp(element["Text"]).vector
            if (vector.shape != (300,)):
                vector = vector.reshape((300,))

            numpy_vector = np.column_stack(vector)
            try:
                cnx.execute("insert into {} (ID_Post,Text_Array,Value) values (?,?,?)"
                            .format(label)
                            ,(index,numpy_vector,int(joined_table.loc[index,"Value"]),))
            except IntegrityError:
                print(index)
            con.commit()








In [33]:
# Fixing the saving of the value as int64 - does nothing if the value is already correctly saved as int.

int64_for_1 = b'\x01\x00\x00\x00\x00\x00\x00\x00'
int64_for_0 = b'\x00\x00\x00\x00\x00\x00\x00\x00'

for label in labels:
    cnx.execute("""
    UPDATE {}
SET Value = 0
WHERE Value = ?;""".format(label),(int64_for_0,))
    con.commit()

for label in labels:
    cnx.execute("""
    UPDATE {}
SET Value = 1
WHERE Value = ?;""".format(label),(int64_for_0,))
    con.commit()


In [36]:
con.close()

In [60]:
con = sqlite3.connect("corpus_NLP_preprocessed", detect_types=sqlite3.PARSE_DECLTYPES)



In [35]:
pd.read_sql_query("SELECT* FROM Inappropriate ",con)

,ID_Post,Text_Array,Value
0,3326,"[[0.8686386, -1.2682943, 0.29122144, -2.601798...",0
1,5321,"[[2.5961683, -0.45668802, -0.6874493, -0.39244...",0
2,5590,"[[1.2644377, -0.07373819, 0.40302694, -1.30055...",0
3,6015,"[[1.7140241, -0.8711243, -0.9298902, -0.980474...",1
4,8213,"[[4.1509, -1.8794401, 1.2093123, 0.3584888, -5...",0
...,...,...,...
3594,1004115,"[[1.0525885, 0.12252, -0.66452307, -0.7557486,...",0
3595,1004189,"[[2.0198832, -0.85422903, 0.27278438, -1.14722...",0
3596,1004571,"[[1.1416737, -0.0017574765, -0.676475, -0.1457...",0
3597,1006462,"[[1.5501649, -0.777564, -0.1885027, -1.0172104...",0


In [24]:
pd.read_sql_query("SELECT* FROM Discriminating",con).loc[762,"Value"]

b'\x01\x00\x00\x00\x00\x00\x00\x00'